# GeoTif Training
- Pick index for tring
- Convert the nasa data to lable tif

### Preparing the data 
- target index list
- check the data count for each index
- check the shapefile
- show the nasa points on map



In [1]:
index_list=[581,1806,1212,1189,438,489,782]

tif_folder_path=r'G:\GithubProject\GoogleEarthEngineTask\Tif_Trainning\tif'
J1V_shapefile_path=r'G:\GithubProject\GoogleEarthEngineTask\Tif_Trainning\nasa_data\J1V\fire_nrt_J1V-C2_536460.shp'

Target_DP_shapefile_path=r'G:\GithubProject\GoogleEarthEngineTask\Tif_Trainning\Target_DP_shape\c1212_ADRSM_T.shp'

In [2]:
import os

def check_tif_data(index_list,tif_path):
    # count the tif file number for files start with index
    for index in index_list:
        tif_file_list=os.listdir(tif_path)
        tif_file_list=[file for file in tif_file_list if file.startswith(str(index))]
        print(f'index {index} has {len(tif_file_list)} tif files')

check_tif_data(index_list,tif_folder_path)


index 581 has 42 tif files
index 1806 has 42 tif files
index 1212 has 42 tif files
index 1189 has 42 tif files
index 438 has 42 tif files
index 489 has 42 tif files
index 782 has 42 tif files


In [5]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
import leafmap
import geopandas as gpd

def show_tif_image_rgb(source_type, file_path):
    if source_type == 'nicfi':
        # read the tif file
        tif_file = rasterio.open(file_path)
        
        # read RGB bands
        blue = tif_file.read(1).astype(float)
        green = tif_file.read(2).astype(float)
        red = tif_file.read(3).astype(float)
        
        def normalize_nicfi(band):
            # Clip values to GEE visualization range
            band = np.clip(band, 64, 5454)  # Match GEE min/max values
            # Normalize to 0-1 range based on the visualization parameters
            band = (band - 64) / (5454 - 64)
            return band
        
        red = normalize_nicfi(red)
        green = normalize_nicfi(green)
        blue = normalize_nicfi(blue)
        
        # stack bands into RGB image
        rgb = np.dstack((red, green, blue))
        
        # Apply gamma correction matching GEE
        gamma = 1.8  # Same gamma as GEE
        rgb = np.power(rgb, 1/gamma)
        
        # get filename for title
        title = os.path.basename(file_path)
        
        # display image
        plt.figure(figsize=(4,4))
        plt.imshow(rgb)
        plt.title(title)
        plt.axis('off')
        plt.show()


def display_shapefile_on_map(shapefile_path):
    """
    Loads a shapefile and displays it on an interactive map.

    Parameters:
    shapefile_path (str): The file path to the shapefile.

    Returns:
    leafmap.Map: An interactive map with the shapefile overlay.
    """
    # Load the shapefile as a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)
    
    red_style = {
        'color': 'red',  # outline color
        'fillOpacity': 0.1,  # transparency of fill
        'weight': 2  # line thickness
    }
    # Calculate the centroid for map centering
    map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
    
    # Create an interactive map centered on the shapefile area
    m = leafmap.Map(center=map_center, zoom=10)
    
    # Get the filename without extension to use as layer name
    layer_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    
    # Add the shapefile as a layer with filename as layer name
    m.add_gdf(gdf, layer_name=layer_name, style=red_style)

    # Add the shapefile as a layer
    m.add_gdf(gdf, layer_name="Shapefile Layer",style=red_style)
    
    # Optionally, add a satellite basemap
    m.add_basemap("SATELLITE")

    return m

In [ ]:
# show the each image from the index list
for index in index_list:
    file_path=os.path.join(tif_folder_path,f'{index}-2021-04-nicfi.tif')
    show_tif_image_rgb('nicfi',file_path)



In [8]:
# show the points on tile map

map_object = display_shapefile_on_map(Target_DP_shapefile_path)
map_object

Map(center=[-10.914215450970215, -75.32548261768909], controls=(ZoomControl(options=['position', 'zoom_in_text…